In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
# Load morphometry data
df_all_cores = pd.read_csv("foram_frag_morphometry.csv", encoding='latin-1')
df_all_cores.head(5)

/var/folders/ml/v613w2ns1z31hpn87j8vcl8h0000gq/T/ipykernel_54477/1635003737.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all_cores = pd.read_csv("foram_frag_morphometry.csv", encoding='latin-1')


,Sample,Species,Type,Diameter,Aream2,Areamm2,Perimeterm,Perimeterm2,ConvexAream2,convexAreamm2,...,Solidity,Roundnessm,Circularity,Mean,Stddev,StddevInvariant,Skewness,Kurtosis,5thmoment,6thmoment
0,BARP9403,Globigerina bulloides,Planktonic Formainifera,292.0,89868.5,0.066854,1189.727051,1.026140,92513.5,0.068821,...,0.971410,0.809051,0.797853,119.732765,17.067535,0.143685,-1.016231,4.345409,-10.348482,34.153194
1,BARP9403,Globigerina bulloides,Planktonic Formainifera,257.0,69480.0,0.051687,1095.401123,0.944784,72929.0,0.054252,...,0.952707,0.684475,0.727651,115.244179,16.724291,0.147724,-1.076683,3.959310,-9.469131,29.717978
2,BARP9403,Globigerina bulloides,Planktonic Formainifera,192.0,38891.0,0.028931,794.867065,0.685573,40312.0,0.029988,...,0.964750,0.924026,0.773517,102.506828,13.562996,0.132553,-1.175333,4.801376,-13.114857,46.299988
3,BARP9403,Globigerinella siphonifera,Planktonic Formainifera,231.0,56545.5,0.042065,1024.045776,0.883240,60057.5,0.044677,...,0.941523,0.723158,0.677593,101.876572,14.474765,0.141386,-0.046323,3.245234,0.432873,18.649073
4,BARP9403,Globigerinella siphonifera,Planktonic Formainifera,350.0,129298.5,0.096186,1513.633667,1.305509,136248.5,0.101356,...,0.948990,0.613363,0.709189,115.152351,15.260325,0.133828,-0.497275,4.133267,-6.165670,28.767853


In [3]:
# Separate foraminifera and fragment data
foram_data = df_all_cores[df_all_cores['Type'] == 'Planktonic Formainifera']
fragment_data = df_all_cores[df_all_cores['Type'] == 'Fragments']

In [4]:
# Drop rows with unknown Core ID
foram_data = foram_data[foram_data["Sample"] != "unknown"]
fragment_data = fragment_data[fragment_data["Sample"] != "unknown"]

In [5]:
# Confirm unique core ids 
foram_data['Sample'].unique()

array(['BARP9403', 'BARP9406', 'BARP9409', 'BARP9412', 'BARP9422',
       'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437 ', 'BARP9439',
       'BARP9441', 'BARP9442', 'BARP9443', 'MDB04-2873',
       'MD04-2875B 0-2 Autolabel and Cleaned', 'MD04-2876',
       'MD04-2877 0-2 Autolable and Cleaned', 'MD12-3418C2', 'MD12-3423',
       'MD 76-011', 'MD76-132', 'MD76-133 ', 'MD76-136',
       'MD76-164 0-2 Autolabel and Cleaned', 'MD77-160 ', 'MD77-169',
       'MD77-171 ', 'MD77-178', 'MD77-180 ', 'MD77-182 ', 'MD77-184 ',
       'MD77-185 0-2 Autolabel and Cleaned', 'MD77-195', 'MD77-197',
       'MD77-200', 'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256',
       'MD79-257', 'MD79-260d', 'MD79-261', 'MD79-275', 'MD79-276',
       'MD79-277 0-2 Autolable and Cleaned', 'MD81-345', 'MD85-640',
       'MD90-0936', 'MD90-0938', 'MD90-0939', 'MD90-0940',
       'MD90-0949 0-1 Autolabel and Cleaned',
       'MD90-O955 0-3 Autolabel and Cleaned',
       'MD90-0956 0-2 Autolabel and Cleaned', 'MD90-

In [6]:
# Strictly retrieve Core Ids only
foram_data['core_id'] = foram_data['Sample'].apply(lambda x: x.split(" ")[0])
fragment_data['core_id'] = fragment_data['Sample'].apply(lambda x: x.split(" ")[0])

In [7]:
# Replace wrong texts with their appropriate Core IDs in the Planktonic Formainifera data
foram_data['core_id'] = (foram_data['core_id'].replace("MD", "MD76-011")
                         .replace("MDB04-2873", "MD04-2873")
                         .replace("MD79-260d", "MD79-260")
                         .replace("BARDP9411", "BARP9411") 
                         .replace("MD90-O955", "MD90-0955") 
                         )


# Confirm string replacements
foram_data['core_id'].unique()

array(['BARP9403', 'BARP9406', 'BARP9409', 'BARP9412', 'BARP9422',
       'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437', 'BARP9439',
       'BARP9441', 'BARP9442', 'BARP9443', 'MD04-2873', 'MD04-2875B',
       'MD04-2876', 'MD04-2877', 'MD12-3418C2', 'MD12-3423', 'MD76-011',
       'MD76-132', 'MD76-133', 'MD76-136', 'MD76-164', 'MD77-160',
       'MD77-169', 'MD77-171', 'MD77-178', 'MD77-180', 'MD77-182',
       'MD77-184', 'MD77-185', 'MD77-195', 'MD77-197', 'MD77-200',
       'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256', 'MD79-257',
       'MD79-260', 'MD79-261', 'MD79-275', 'MD79-276', 'MD79-277',
       'MD81-345', 'MD85-640', 'MD90-0936', 'MD90-0938', 'MD90-0939',
       'MD90-0940', 'MD90-0949', 'MD90-0955', 'MD90-0956', 'MD90-0957',
       'MD90-0958', 'MD90-0959', 'MD90-0960', 'MD90-0961', 'MD90-0963',
       'MD96-2044', 'MD96-2045', 'MD96-2049', 'MD96-2051', 'MD96-2053',
       'MD96-2054', 'MD96-2055', 'MD96-2056', 'MD96-2058', 'MD96-2059',
       'MD96-2060', 'MD96-2061

In [ ]:
# Replace wrong texts with their appropriate Core IDs in the fragments data
fragment_data['core_id'] = (fragment_data['core_id'].replace("MD", "MD76-011")
                         .replace("MDB04-2873", "MD04-2873")
                         .replace("MD79-260d", "MD79-260")
                         .replace("BARDP9411", "BARP9411") 
                         .replace("MD90-O955", "MD90-0955") 
                         )

# Confirm string replacements
fragment_data['core_id'].unique()

array(['BARP9411', 'BARP9403', 'BARP9406', 'BARP9409', 'BARP9412',
       'BARP9422', 'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437',
       'BARP9439', 'BARP9441', 'BARP9442', 'BARP9443', 'MD04-2873',
       'MD04-2875B', 'MD04-2876', 'MD04-2877', 'MD12-3418C2', 'MD12-3423',
       'MD76-011', 'MD76-132', 'MD76-133', 'MD76-136', 'MD76-164',
       'MD77-160', 'MD77-169', 'MD77-171', 'MD77-178', 'MD77-180',
       'MD77-182', 'MD77-184', 'MD77-185', 'MD77-195', 'MD77-197',
       'MD77-200', 'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256',
       'MD79-257', 'MD79-260', 'MD79-261', 'MD79-275', 'MD79-276',
       'MD79-277', 'MD81-345', 'MD85-640', 'MD90-0936', 'MD90-0938',
       'MD90-0939', 'MD90-0940', 'MD90-0949', 'MD90-0955', 'MD90-0956',
       'MD90-0957', 'MD90-0958', 'MD90-0959', 'MD90-0960', 'MD90-0961',
       'MD90-0963', 'MD96-2044', 'MD96-2045', 'MD96-2049', 'MD96-2051',
       'MD96-2053', 'MD96-2054', 'MD96-2055', 'MD96-2056', 'MD96-2058',
       'MD96-2059', 'MD96-2060'

In [11]:
# Load data containing target Core IDs
df_target_cores = pd.read_csv("dissolution_manuscript_final.csv")

In [12]:
# Extract target core ids
target_cores = df_target_cores.iloc[ : , 0]
target_cores = list(target_cores)

In [13]:
# Create new planktonic foraminifera DataFrame based on target Core IDs
df_foram = foram_data[foram_data['core_id'].isin(target_cores)]

In [93]:
# Create new fragment DataFrame based on target Core IDs
df_fragment = fragment_data[fragment_data['core_id'].isin(target_cores)]

In [14]:
# Confirm no target core is missing
missing_target_cores = []

missing_target_cores.extend(
    core for core in target_cores if core not in df_foram['core_id'].unique()
)

# Show list of missing target cores
print(missing_target_cores) # Should return an empty list if no core is missing

[]


In [94]:
# Group each data by sample (Core ID)
forams_grouped = df_foram.groupby('core_id')
fragments_grouped = fragment_data.groupby('core_id')

## Run Vif (multicollinearity test)

### Forams

In [225]:
def create_vif_data():
    all_features = []  # List to store features for each sample

    for sample in df_foram['core_id'].unique():
        # Forams metrics
        foram_data = forams_grouped.get_group(sample)
        variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
        diameter_forams = foram_data['Diameter'].mean()
        circularity_forams = foram_data['Circularity'].mean()
        solidity_forams = foram_data['Solidity '].mean()
        eccentricity_forams = foram_data['Eccentricity'].mean()
        area_forams = foram_data['Areamm2'].mean()
        perimeter_forams = foram_data['Perimeterm2'].mean()
        roundness_forams = foram_data['Roundnessm'].mean()
        kurtosis_forams = foram_data['Kurtosis'].mean()

        # Append the feature list for the current sample
        all_features.append([variance_forams, circularity_forams, eccentricity_forams,
             perimeter_forams,  kurtosis_forams
        ])

    # Create DataFrame after collecting all features
    vif_data = pd.DataFrame(all_features, columns=['variance_forams', 'circularity_forams', 'eccentricity_forams',
             'perimeter_forams', 'kurtosis_forams'
        
    ])

    return vif_data

In [219]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

def calculate_vif(df, features):
    """Calculates VIF for specified features in a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the features.
        features (list): List of feature names for VIF calculation.

    Returns:
        pd.DataFrame: VIF results for each feature.
    """

    X = df[features]  # Select features for VIF
    X = X.dropna()  # Drop rows with missing values


    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create a DataFrame with the standardized data
    X_scaled_df = pd.DataFrame(X_scaled, columns=features)

    # Calculate VIF for each feature
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_scaled_df.columns
    vif_data['VIF'] = [variance_inflation_factor(X_scaled_df.values, i) for i in range(X_scaled_df.shape[1])]

    return vif_data

In [246]:
df_X = create_vif_data()
feature = ['variance_forams', 'circularity_forams', 'eccentricity_forams',
        'perimeter_forams', 'kurtosis_forams'
        ]

calculate_vif(df_X, feature)

,Feature,VIF
0,variance_forams,4.359510
1,circularity_forams,1.945695
2,eccentricity_forams,1.689598
3,perimeter_forams,3.267246
4,kurtosis_forams,2.112801


## Fragments

In [244]:
def create_frag_vif_data():
    all_features = []  # List to store features for each sample

    for sample in df_fragment['core_id'].unique():
        # Forams metrics
        fragment_data = fragments_grouped.get_group(sample)
        variance_frag = np.var(fragment_data['Diameter'], ddof=1)  # Sample variance
        diameter_frag = fragment_data['Diameter'].mean()
        circularity_frag = fragment_data['Circularity'].mean()
        solidity_frag = fragment_data['Solidity '].mean()
        eccentricity_frag = fragment_data['Eccentricity'].mean()
        area_frag = fragment_data['Areamm2'].mean()
        perimeter_frag = fragment_data['Perimeterm2'].mean()
        roundness_frag = fragment_data['Roundnessm'].mean()
        kurtosis_frag = fragment_data['Kurtosis'].mean()

        # Append the feature list for the current sample
        all_features.append([variance_frag, diameter_frag, circularity_frag,
              eccentricity_frag, kurtosis_frag
        ])

    # Create DataFrame after collecting all features
    frag_vif_data = pd.DataFrame(all_features, columns=[
        'variance_frag', 'diameter_frag', 'circularity_frag',
              'eccentricity_frag',   'kurtosis_frag',
            
    ])

    return frag_vif_data

In [245]:
df_frag_X = create_frag_vif_data()
feature = ['variance_frag', 'diameter_frag', 'circularity_frag', 
              'eccentricity_frag', 'kurtosis_frag',]

calculate_vif(df_frag_X, feature)

,Feature,VIF
0,variance_frag,2.173559
1,diameter_frag,2.530585
2,circularity_frag,2.265683
3,eccentricity_frag,3.318253
4,kurtosis_frag,1.501188


## Forams + Fragments

In [268]:
def create_full_vif_data():
    all_features = []  # List to store features for each core/sample

    # Get the common core IDs present in both datasets
    common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

    for core_id in common_core_ids:
        # Forams metrics
        foram_data = forams_grouped.get_group(core_id)
        variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
        diameter_forams = foram_data['Diameter'].mean()
        circularity_forams = foram_data['Circularity'].mean()
        solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
        eccentricity_forams = foram_data['Eccentricity'].mean()
        area_forams = foram_data['Areamm2'].mean()
        perimeter_forams = foram_data['Perimeterm2'].mean()
        roundness_forams = foram_data['Roundnessm'].mean()
        kurtosis_forams = foram_data['Kurtosis'].mean()
        
        # Fragments metrics
        fragment_data = fragments_grouped.get_group(core_id)
        variance_frag = np.var(fragment_data['Diameter'], ddof=1)
        diameter_frag = fragment_data['Diameter'].mean()
        circularity_frag = fragment_data['Circularity'].mean()
        solidity_frag = fragment_data['Solidity '].mean()
        eccentricity_frag = fragment_data['Eccentricity'].mean()
        area_frag = fragment_data['Areamm2'].mean()
        perimeter_frag = fragment_data['Perimeterm2'].mean()
        roundness_frag = fragment_data['Roundnessm'].mean()
        kurtosis_frag = fragment_data['Kurtosis'].mean()

        # Append combined features
        all_features.append([
            variance_forams, circularity_forams, eccentricity_forams,
             perimeter_forams, kurtosis_forams,
            variance_frag, circularity_frag, eccentricity_frag,
            perimeter_frag,  kurtosis_frag
        ])

    # Create DataFrame after collecting all features
    full_vif_data = pd.DataFrame(all_features, columns=[
        'variance_forams', 'circularity_forams',
        'eccentricity_forams', 'perimeter_forams',  'kurtosis_forams',
        'variance_frag', 'circularity_frag', 
        'eccentricity_frag',  'perimeter_frag',  'kurtosis_frag'
    ])
    
    return full_vif_data


In [269]:
df_full_X = create_full_vif_data()
feature = ['variance_forams', 'circularity_forams', 
        'eccentricity_forams',  'perimeter_forams', 'kurtosis_forams',
        'variance_frag',  'circularity_frag', 
        'eccentricity_frag', 'perimeter_frag',  'kurtosis_frag']

calculate_vif(df_full_X, feature)

,Feature,VIF
0,variance_forams,6.591739
1,circularity_forams,4.116821
2,eccentricity_forams,2.122116
3,perimeter_forams,9.638905
4,kurtosis_forams,3.042753
5,variance_frag,3.222646
6,circularity_frag,5.482696
7,eccentricity_frag,4.294791
8,perimeter_frag,7.219916
9,kurtosis_frag,4.213382


In [284]:
def calculate_fv_index():
    fv_index_results = []  # List to store FV-Index metrics for each sample

    # Get the common core IDs present in both datasets
    common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

    for core_id in common_core_ids:

        # Forams metrics
        foram_data = forams_grouped.get_group(core_id)
        variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
        # diameter_forams = foram_data['Diameter'].mean()
        circularity_forams = foram_data['Circularity'].mean()
        # solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
        eccentricity_forams = foram_data['Eccentricity'].mean()
        # area_forams = foram_data['Areamm2'].mean()
        # perimeter_forams = foram_data['Perimeterm2'].mean()
        # roundness_forams = foram_data['Roundnessm'].mean()
        kurtosis_forams = foram_data['Kurtosis'].mean()
        
        # Fragments metrics
        fragment_data = fragments_grouped.get_group(core_id)
        variance_frag = np.var(fragment_data['Diameter'], ddof=1)
        # diameter_frag = fragment_data['Diameter'].mean()
        # circularity_frag = fragment_data['Circularity'].mean()
        # solidity_frag = fragment_data['Solidity '].mean()
        eccentricity_frag = fragment_data['Eccentricity'].mean()
        # area_frag = fragment_data['Areamm2'].mean()
        # perimeter_frag = fragment_data['Perimeterm2'].mean()
        # roundness_frag = fragment_data['Roundnessm'].mean()
        # kurtosis_frag = fragment_data['Kurtosis'].mean()


        # Compute FV-Index
        if variance_forams > 0 and circularity_forams > 0:  # Avoid division by zero
            fv_index = (variance_frag / variance_forams) * ((1-eccentricity_frag)/ circularity_forams) * (eccentricity_forams/kurtosis_forams)
        else:
            fv_index = np.nan  # Set to NaN if calculations are not valid

        # Store the results
        fv_index_results.append({
            'Core ID': core_id,
            'FV-Index': fv_index
        })

    FV_index = pd.DataFrame(fv_index_results, columns=['Core ID', 'FV-Index'])

    return FV_index


In [285]:
calculate_fv_index()

,Core ID,FV-Index
0,MD90-0959,0.082321
1,MD90-0956,0.107461
2,MD90-0957,0.101096
3,MD96-2056,0.064659
4,BARP9439,0.098408
...,...,...
57,MD76-011,0.076462
58,MD76-133,0.063243
59,MD96-2067a,0.042836
60,MD90-0960,0.076423


In [286]:
# Load second dataset
df_manu = pd.read_csv("dissolution_manuscript_final.csv")

df_manu.head(5)

,Core ID,Latitude,Longitude,Depth (m),FV-Index,BP-Index,Frag_rate (%),Frag_Intensity,Ω Calcite (CO3 2-),Sp_richness,Sp_diversity,SST (°C),"Thermal Gradient ∆T (T0-200, °C)","Temp_seasonality (Sum-win, °C)",Logpp,Sal (psu)
0,MD96-2045,-31.50,36.75,3047,0.65,1.08,8,0.33,1.17,27,2.27,22.38,5.23,5.743,2.66,35.52
1,MD96-2044,-31.00,45.17,2198,0.87,1.02,4,0.15,1.25,27,2.33,21.44,4.97,5.598,2.61,35.57
2,MD79-276,-30.76,48.26,3927,0.81,1.04,13,0.46,0.88,24,2.02,22.06,6.14,4.681,2.56,35.55
3,MD79-261,-29.31,33.15,2060,1.13,1.02,2,0.01,1.31,22,1.98,24.14,6.43,4.506,2.63,35.39
4,MD79-260,-26.27,36.49,2061,1.08,0.99,2,0.03,1.28,23,2.05,24.75,7.72,4.196,2.56,35.41


In [287]:
fv_data = calculate_fv_index()

In [296]:
pd.set_option('display.max_colwidth', 300)

print(df_manu.merge(fv_data, how='left', on='Core ID').to_string())

# df_manu.merge(fv_data, how='left', on='Core ID')

        Core ID  Latitude  Longitude  Depth (m)  FV-Index_x  BP-Index  Frag_rate (%)  Frag_Intensity  Ω Calcite (CO3 2-)  Sp_richness  Sp_diversity  SST (°C)  Thermal Gradient ∆T (T0-200, °C)  Temp_seasonality (Sum-win, °C)  Logpp  Sal (psu)  FV-Index_y
0     MD96-2045   -31.500      36.75       3047        0.65      1.08              8            0.33                1.17           27          2.27     22.38                              5.23                           5.743   2.66      35.52    0.076502
1     MD96-2044   -31.000      45.17       2198        0.87      1.02              4            0.15                1.25           27          2.33     21.44                              4.97                           5.598   2.61      35.57    0.076415
2      MD79-276   -30.760      48.26       3927        0.81      1.04             13            0.46                0.88           24          2.02     22.06                              6.14                           4.681   2.56      35

In [ ]:
df_manu[df_manu['FV-Index', 'FV-Index']]

KeyError: ('FV-Index_x', 'FV-Index')